In [10]:
import time
import math
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [11]:
ROBOT_CONFIG = {
    'eixo_x': '/CARRO_v1_Slider_2',  
    'eixo_y': '/MESA_v1_Slider_1',
    'eixo_z': '/CASTELO_v1_Slider_3'
}

In [12]:
# 1. Cria a conexão com o cliente ZMQ
client = RemoteAPIClient()
sim = client.require('sim')

# Dicionário para guardar os IDs numéricos (Handles)
handles = {}

def conectar_e_mapear():
    print("🔌 Tentando conectar e buscar juntas...")
    try:
        # Testa conexão pegando o tempo
        t = sim.getSimulationTime()
        print(f"   Conexão ZMQ estabelecida! (Tempo Sim: {t:.2f}s)")
        
        # Loop para buscar cada junta definida
        for eixo, caminho in ROBOT_CONFIG.items():
            try:
                # Pede ao Coppelia o ID do objeto
                h = sim.getObject(caminho)
                handles[eixo] = h
                print(f"   Mapeado: {eixo} -> ID {h}")
            except Exception:
                print(f"   ❌ ERRO: Não encontrei o objeto '{caminho}' na cena.")
                return False
        
        print("\n✅ Setup Completo! O robô está pronto para receber comandos.")
        return True

    except Exception as e:
        print(f"❌ Falha na conexão. O CoppeliaSim está aberto? \nErro: {e}")
        return False

# Executa a função
setup_ok = conectar_e_mapear()

🔌 Tentando conectar e buscar juntas...
   Conexão ZMQ estabelecida! (Tempo Sim: 193.40s)
   Mapeado: eixo_x -> ID 20
   Mapeado: eixo_y -> ID 17
   Mapeado: eixo_z -> ID 23

✅ Setup Completo! O robô está pronto para receber comandos.


In [13]:
def iniciar_simulacao():
    """Inicia a física do simulador"""
    sim.startSimulation()
    print("▶️ Simulação INICIADA.")

def parar_simulacao():
    """Para a física do simulador"""
    sim.stopSimulation()
    print("⏹️ Simulação PARADA.")

def mover_eixo(eixo_nome, posicao_mm):
    """
    Move um eixo específico para uma posição em milímetros.
    eixo_nome: 'eixo_x', 'eixo_y' ou 'eixo_z'
    posicao_mm: valor em milímetros (ex: 150)
    """
    if eixo_nome not in handles:
        print(f"⚠️ Eixo '{eixo_nome}' não existe no mapa.")
        return

    # Converte mm para metros (unidade padrão do Coppelia)
    posicao_metros = posicao_mm / 1000.0
    
    # Envia o comando
    handle_id = handles[eixo_nome]
    sim.setJointTargetPosition(handle_id, posicao_metros)
    print(f"   Movendo {eixo_nome} para {posicao_mm}mm")

def ir_para_home():
    """Zera todos os eixos"""
    print("🏠 Indo para Home (0,0,0)...")
    mover_eixo('eixo_z', 0) # Z primeiro ou por último? Depende da segurança.
    mover_eixo('eixo_x', 0)
    mover_eixo('eixo_y', 0)

def configurar_velocidade(eixo_nome, velocidade_m_s):
    """
    Define a velocidade máxima permitida para o eixo.
    velocidade_m_s: Velocidade em Metros por Segundo (ex: 0.05)
    """
    if eixo_nome not in handles:
        return
        
    handle = handles[eixo_nome]
    
    # O parâmetro 'sim.jointfloatparam_upper_limit' (ID 2017) controla o limite de velocidade
    sim.setObjectFloatParam(handle, sim.jointfloatparam_upper_limit, velocidade_m_s)
    

In [17]:
if setup_ok:
    # 1. Inicia
    iniciar_simulacao()
    
    # Pequena pausa para garantir que o motor de física acordou
    time.sleep(1)

    # 2. Sequência de movimentos (Exemplo CNC)
    # Levanta Z (segurança)
    mover_eixo('eixo_z', 50) 
    time.sleep(2)
    
    # Move X e Y
    mover_eixo('eixo_x', 120)
    mover_eixo('eixo_y', 80)
    time.sleep(2)
    
    # Volta X
    mover_eixo('eixo_x', 0)
    time.sleep(2)
    
    # 3. Finaliza
    ir_para_home()
    time.sleep(1)
    parar_simulacao()
else:
    print("⚠️ Rode a Célula 3 novamente para corrigir a conexão.")

▶️ Simulação INICIADA.
   Movendo eixo_z para 50mm
   Movendo eixo_x para 120mm
   Movendo eixo_y para 80mm
   Movendo eixo_x para 0mm
🏠 Indo para Home (0,0,0)...
   Movendo eixo_z para 0mm
   Movendo eixo_x para 0mm
   Movendo eixo_y para 0mm
⏹️ Simulação PARADA.
